In [11]:
from pathlib import Path
path = Path('/mnt/c/Users/OuKu/Developments/MobyLe/data/nl_veenweiden_s1_asc_t088/')

line_range = (2000, 2100)
pixel_range = (5000, 5100)

In [12]:
# Get shape of the SLCs
shape = []
for file in ['lines', 'pixels']:
    with open(path / 'n{}_crp.txt'.format(file)) as f:
        content = f.readlines()
        shape.append(int(content[0]))
shape = tuple(shape)
print(shape)

(9914, 41174)


In [13]:
dir_slcs = [d / 'h2ph_nl_veenweiden.raw' for d in path.iterdir() if d.is_dir()]

print(dir_slcs)


[PosixPath('/mnt/c/Users/OuKu/Developments/MobyLe/data/nl_veenweiden_s1_asc_t088/20210407/h2ph_nl_veenweiden.raw'), PosixPath('/mnt/c/Users/OuKu/Developments/MobyLe/data/nl_veenweiden_s1_asc_t088/20210413/h2ph_nl_veenweiden.raw'), PosixPath('/mnt/c/Users/OuKu/Developments/MobyLe/data/nl_veenweiden_s1_asc_t088/20210419/h2ph_nl_veenweiden.raw')]


In [14]:
n_time = len(dir_slcs)
n_points = (line_range[1]-line_range[0]) * (pixel_range[1]-pixel_range[0])

print(n_time)
print(n_points)

3
10000


In [15]:
import numpy as np
import xarray as xr

stack=np.empty((n_points, 0))
for d in dir_slcs:
    slc = np.memmap(d, mode='r', dtype='float32',shape=shape)
    stack = np.append(stack, slc[line_range[0]:line_range[1], pixel_range[0]:pixel_range[1]].reshape(n_points, 1), axis=1)

print(stack)
print(stack.shape)

[[-6.50455986e-05 -8.64321846e-05 -2.26245014e-04]
 [-6.50449219e-05 -8.64314788e-05 -2.26242439e-04]
 [-6.50442380e-05 -8.64307731e-05 -2.26239863e-04]
 ...
 [-6.49962167e-05 -8.63887981e-05 -2.26062431e-04]
 [-6.49955764e-05 -8.63881214e-05 -2.26059987e-04]
 [-6.49949288e-05 -8.63874448e-05 -2.26057527e-04]]
(10000, 3)


In [16]:
data_xr = xr.Dataset(data_vars=dict(h2ph=(["points", "time"], stack),),
                     coords=dict(
                        points=np.arange(n_points),
                        time=np.arange(n_time),
                        ))
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    h2ph     (points, time) float64 -6.505e-05 -8.643e-05 ... -0.0002261

In [17]:
import stmat
data_xr = data_xr.stmat.add_metadata({"name":"test_stm"})

In [18]:
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    h2ph     (points, time) float64 -6.505e-05 -8.643e-05 ... -0.0002261
Attributes:
    name:     test_stm